In [1]:
import dask.dataframe as dd
from os.path import join
import os
import pandas as pd
import random
import numpy as np
from dask.distributed import Client

# Assuming admission and concept are your DataFrames

# Step 1: Set up a Dask client
client = Client()

In [2]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 31.71 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62637,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 31.71 GiB
Comm: tcp://127.0.0.1:62661,Total threads: 2
Dashboard: http://127.0.0.1:62663/status,Memory: 7.93 GiB
Nanny: tcp://127.0.0.1:62640,


In [140]:
concepts_path = "C:\\Users\\fjn197\\PhD\\projects\\PHAIR\\pipelines\\ehr_preprocess\\ehr_preprocess\\formatted_data\\synthea500_test\\concept.medication.csv"
concepts = pd.read_csv(concepts_path)
# sample n patients
rand_pats = np.random.choice(concepts['PID'].unique().tolist(), 100, replace=False)

concepts = concepts[concepts.PID.isin(rand_pats)]
concepts = concepts.sort_values(by=['PID','TIMESTAMP'])
concepts = concepts.drop(columns=['ADMISSION_ID'])
concepts['TIMESTAMP'] = pd.to_datetime(concepts['TIMESTAMP'])

In [141]:
concepts['DIFF'] = concepts.groupby('PID')['TIMESTAMP'].diff()
concepts['NEW_SEGMENT'] = concepts['DIFF'] > pd.Timedelta(days=7) # 3 days is a rough estimation of a new segment
concepts['SEGMENT'] = concepts.groupby('PID', group_keys=False)['NEW_SEGMENT'].apply(lambda x: x.astype(int).cumsum()) + 1
concepts = concepts.drop(columns=['DIFF', 'NEW_SEGMENT'])
concepts = concepts.sort_values(by=['PID','TIMESTAMP'])
concepts['ADMISSION_ID'] = concepts['PID'].astype(str) + '-' + concepts['SEGMENT'].astype(str)
concepts = concepts.drop(columns=['SEGMENT'])

In [142]:
# get first and last timestamp for each ADMISSION_ID
first = concepts.groupby('ADMISSION_ID')['TIMESTAMP'].first().reset_index()
last = concepts.groupby('ADMISSION_ID')['TIMESTAMP'].last().reset_index()
# Store it in a dataframe with ADMISISION_ID,PID, TIMESTAMP, TIMESTAMP_END
admission = pd.merge(first, last, on='ADMISSION_ID', suffixes=('_START', '_END'))
admission = admission.rename(columns={'TIMESTAMP_START': 'TIMESTAMP', 'TIMESTAMP_END': 'TIMESTAMP_END'})
admission = admission.sort_values(by=['ADMISSION_ID','TIMESTAMP'])
admission['PID'] = admission['ADMISSION_ID'].str.split('-').str[:-1].str.join('-')
concepts = concepts.drop(columns=['ADMISSION_ID'])


In [158]:
admission

,ADMISSION_ID,TIMESTAMP,TIMESTAMP_END,PID
0,03aaf341-6896-f8ba-3036-b3b7eef21f7f-1,2006-03-13 12:47:18+00:00,2006-03-13 12:47:18+00:00,03aaf341-6896-f8ba-3036-b3b7eef21f7f
1,03aaf341-6896-f8ba-3036-b3b7eef21f7f-10,2014-03-31 18:58:14+00:00,2014-03-31 18:58:14+00:00,03aaf341-6896-f8ba-3036-b3b7eef21f7f
2,03aaf341-6896-f8ba-3036-b3b7eef21f7f-11,2014-04-28 12:47:18+00:00,2014-04-28 12:47:18+00:00,03aaf341-6896-f8ba-3036-b3b7eef21f7f
3,03aaf341-6896-f8ba-3036-b3b7eef21f7f-12,2015-02-10 14:37:39+00:00,2015-02-10 14:37:39+00:00,03aaf341-6896-f8ba-3036-b3b7eef21f7f
4,03aaf341-6896-f8ba-3036-b3b7eef21f7f-13,2015-05-04 12:47:18+00:00,2015-05-04 12:47:18+00:00,03aaf341-6896-f8ba-3036-b3b7eef21f7f
...,...,...,...,...
2996,fe4c527b-242b-19f7-5e29-97ac15ff7b35-95,1993-03-12 10:49:56+00:00,1993-03-26 10:49:56+00:00,fe4c527b-242b-19f7-5e29-97ac15ff7b35
2997,fe4c527b-242b-19f7-5e29-97ac15ff7b35-96,1993-11-12 10:49:56+00:00,1993-11-12 10:49:56+00:00,fe4c527b-242b-19f7-5e29-97ac15ff7b35
2998,fe4c527b-242b-19f7-5e29-97ac15ff7b35-97,1994-03-11 10:49:56+00:00,1994-03-18 10:49:56+00:00,fe4c527b-242b-19f7-5e29-97ac15ff7b35
2999,fe4c527b-242b-19f7-5e29-97ac15ff7b35-98,1994-05-06 10:49:56+00:00,1994-05-06 10:49:56+00:00,fe4c527b-242b-19f7-5e29-97ac15ff7b35


In [143]:
n_partitions = 10
# Define a custom function to find the ADMISSION_ID based on the TIMESTAMP
concepts_dask = dd.from_pandas(concepts, npartitions=n_partitions)
concepts_dask = concepts_dask.set_index('PID')
admission_dask = dd.from_pandas(admission, npartitions=n_partitions)
admission_dask = admission_dask.set_index('PID')

In [ ]:
def assign_admission_id(concepts, admission):
    merged_df = concepts.merge(admission, how='inner', left_index=True, right_index=True, suffixes=('_CONCEPT', '_ADMISSION'))
    

In [133]:
concepts_dask['TIMESTAMP'] = dd.to_datetime(concepts_dask['TIMESTAMP'])
admission_dask['TIMESTAMP'] = dd.to_datetime(admission_dask['TIMESTAMP'])
admission_dask['TIMESTAMP_END'] = dd.to_datetime(admission_dask['TIMESTAMP_END'])

In [154]:
merged_df = concepts_dask.merge(admission_dask,how='inner', left_index=True, right_index=True, suffixes=('_CONCEPT', '_ADMISSION'))

In [155]:
merged_df = merged_df[(merged_df['TIMESTAMP_CONCEPT'] >= merged_df['TIMESTAMP_ADMISSION']) & (merged_df['TIMESTAMP_CONCEPT'] <= merged_df['TIMESTAMP_END'])]

In [156]:
merged_df = merged_df.compute()

In [ ]:
merged_df

,TIMESTAMP_CONCEPT,CONCEPT,ADMISSION_ID,TIMESTAMP_ADMISSION,TIMESTAMP_END
PID,,,,,
03aaf341-6896-f8ba-3036-b3b7eef21f7f,2021-06-07 12:47:18+00:00,314076,03aaf341-6896-f8ba-3036-b3b7eef21f7f-20,2021-06-07 12:47:18+00:00,2021-06-07 12:47:18+00:00
03aaf341-6896-f8ba-3036-b3b7eef21f7f,2017-05-22 12:47:18+00:00,705129,03aaf341-6896-f8ba-3036-b3b7eef21f7f-16,2017-05-15 12:47:18+00:00,2017-05-22 12:47:18+00:00
03aaf341-6896-f8ba-3036-b3b7eef21f7f,2009-03-30 12:47:18+00:00,314076,03aaf341-6896-f8ba-3036-b3b7eef21f7f-4,2009-03-30 12:47:18+00:00,2009-03-30 12:47:18+00:00
03aaf341-6896-f8ba-3036-b3b7eef21f7f,2022-04-29 01:25:38+00:00,1719286,03aaf341-6896-f8ba-3036-b3b7eef21f7f-22,2022-04-25 13:25:38+00:00,2022-05-02 01:25:38+00:00
03aaf341-6896-f8ba-3036-b3b7eef21f7f,2022-04-28 13:25:38+00:00,1719286,03aaf341-6896-f8ba-3036-b3b7eef21f7f-22,2022-04-25 13:25:38+00:00,2022-05-02 01:25:38+00:00
...,...,...,...,...,...
fe4c527b-242b-19f7-5e29-97ac15ff7b35,1994-06-10 10:49:56+00:00,308136,fe4c527b-242b-19f7-5e29-97ac15ff7b35-99,1994-06-10 10:49:56+00:00,1994-06-10 10:49:56+00:00
fe4c527b-242b-19f7-5e29-97ac15ff7b35,1994-05-06 10:49:56+00:00,308136,fe4c527b-242b-19f7-5e29-97ac15ff7b35-98,1994-05-06 10:49:56+00:00,1994-05-06 10:49:56+00:00
fe4c527b-242b-19f7-5e29-97ac15ff7b35,1994-05-06 10:49:56+00:00,310798,fe4c527b-242b-19f7-5e29-97ac15ff7b35-98,1994-05-06 10:49:56+00:00,1994-05-06 10:49:56+00:00


2023-05-27 23:50:33,338 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:62658', name: 1, status: running, memory: 0, processing: 0>
2023-05-27 23:50:33,353 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:62662', name: 2, status: running, memory: 0, processing: 0>
2023-05-27 23:50:33,431 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:62658'.
2023-05-27 23:50:33,438 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:62662'.
2023-05-27 23:50:36,556 - distributed.nanny - WARNING - Restarting worker
2023-05-27 23:50:36,588 - distributed.nanny - WARNING - Restarting worker


In [153]:
merged_df.index.nunique()

100

In [107]:
def assign_admission_id(row):
    if row['TIMESTAMP'] >= row['TIMESTAMP_admission'] and row['TIMESTAMP'] <= row['TIMESTAMP_END']:
        return row['ADMISSION_ID']
    else:
        return None

In [108]:
merged_df['ADMISSION_ID_concept'] = merged_df.apply(assign_admission_id, axis=1, meta=('ADMISSION_ID', 'object'))

In [110]:
merged_df = merged_df.drop(['ADMISSION_ID', 'TIMESTAMP_admission', 'TIMESTAMP_END'], axis=1)
merged_df = merged_df.rename(columns={'ADMISSION_ID_concept': 'ADMISSION_ID'})

In [114]:
admission

,ADMISSION_ID,TIMESTAMP,TIMESTAMP_END,PID
0,04385df7-38db-e273-5327-74d2144acf82-1,2022-10-27 21:51:03+00:00,2022-10-27 21:51:03+00:00,04385df7-38db-e273-5327-74d2144acf82
1,04f265eb-105d-7819-5fd8-5e9558c4e8f6-1,2015-08-10 11:08:21+00:00,2015-08-10 11:08:21+00:00,04f265eb-105d-7819-5fd8-5e9558c4e8f6
2,04f265eb-105d-7819-5fd8-5e9558c4e8f6-10,2020-10-02 10:55:55+00:00,2020-10-02 10:55:55+00:00,04f265eb-105d-7819-5fd8-5e9558c4e8f6
3,04f265eb-105d-7819-5fd8-5e9558c4e8f6-11,2020-12-28 10:55:55+00:00,2020-12-28 10:55:55+00:00,04f265eb-105d-7819-5fd8-5e9558c4e8f6
4,04f265eb-105d-7819-5fd8-5e9558c4e8f6-12,2021-01-04 10:55:55+00:00,2021-01-04 10:55:55+00:00,04f265eb-105d-7819-5fd8-5e9558c4e8f6
...,...,...,...,...
3002,fe4c527b-242b-19f7-5e29-97ac15ff7b35-95,1984-02-03 10:49:56+00:00,1984-02-03 10:49:56+00:00,fe4c527b-242b-19f7-5e29-97ac15ff7b35
3003,fe4c527b-242b-19f7-5e29-97ac15ff7b35-96,1984-11-23 10:49:56+00:00,1984-11-23 10:49:56+00:00,fe4c527b-242b-19f7-5e29-97ac15ff7b35
3004,fe4c527b-242b-19f7-5e29-97ac15ff7b35-97,1985-01-25 10:49:56+00:00,1985-01-25 10:49:56+00:00,fe4c527b-242b-19f7-5e29-97ac15ff7b35
3005,fe4c527b-242b-19f7-5e29-97ac15ff7b35-98,1985-03-29 10:49:56+00:00,1985-03-29 10:49:56+00:00,fe4c527b-242b-19f7-5e29-97ac15ff7b35


In [113]:
merged_df.head(10)

,TIMESTAMP,CONCEPT,SEGMENT,ADMISSION_ID_admission,ADMISSION_ID
PID,,,,,
04385df7-38db-e273-5327-74d2144acf82,2022-10-27 21:51:03+00:00,562251,1,04385df7-38db-e273-5327-74d2144acf82-1,04385df7-38db-e273-5327-74d2144acf82-1
04f265eb-105d-7819-5fd8-5e9558c4e8f6,2018-05-28 12:49:55+00:00,849574,3,04f265eb-105d-7819-5fd8-5e9558c4e8f6-1,None
04f265eb-105d-7819-5fd8-5e9558c4e8f6,2018-05-28 12:49:55+00:00,849574,3,04f265eb-105d-7819-5fd8-5e9558c4e8f6-10,None
04f265eb-105d-7819-5fd8-5e9558c4e8f6,2018-05-28 12:49:55+00:00,849574,3,04f265eb-105d-7819-5fd8-5e9558c4e8f6-11,None
04f265eb-105d-7819-5fd8-5e9558c4e8f6,2018-05-28 12:49:55+00:00,849574,3,04f265eb-105d-7819-5fd8-5e9558c4e8f6-12,None
04f265eb-105d-7819-5fd8-5e9558c4e8f6,2018-05-28 12:49:55+00:00,849574,3,04f265eb-105d-7819-5fd8-5e9558c4e8f6-13,None
04f265eb-105d-7819-5fd8-5e9558c4e8f6,2018-05-28 12:49:55+00:00,849574,3,04f265eb-105d-7819-5fd8-5e9558c4e8f6-14,None
04f265eb-105d-7819-5fd8-5e9558c4e8f6,2018-05-28 12:49:55+00:00,849574,3,04f265eb-105d-7819-5fd8-5e9558c4e8f6-15,None
04f265eb-105d-7819-5fd8-5e9558c4e8f6,2018-05-28 12:49:55+00:00,849574,3,04f265eb-105d-7819-5fd8-5e9558c4e8f6-2,None


In [181]:

def find_admission_id(row):
    pid = row['PID']
    timestamp = row['TIMESTAMP']
    # also add small timedelta
    td = pd.Timedelta(hours=1)
    matching_admission = admission_dask[(admission_dask['PID'] == pid) \
                                   & ((admission_dask['TIMESTAMP'] -td) <= timestamp) \
                                   & ((admission_dask['TIMESTAMP_END']+td) >= timestamp)]
    if not matching_admission.empty:
        return matching_admission.iloc[0]['ADMISSION_ID']
    else:
        return None

# Apply the custom function to the concept dataframe
concept_dask['ADMISSION_ID'] = concept_dask.apply(find_admission_id, axis=1)
# concept_dask = concept_dask.sort_values(by=['ADMISSION_ID','TIMESTAMP'])
# concept_dask = concept_dask.reset_index(drop=True)

ValueError: Metadata inference failed in `apply`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
AttributeNotImplementedError('Checking whether a Dask DataFrame has any rows may be expensive. However, checking the number of columns is fast. Depending on which of these results you need, use either `len(df.index) == 0` or `len(df.columns) == 0`')

Traceback:
---------
  File "c:\Users\fjn197\Miniconda3\envs\datascience\lib\site-packages\dask\dataframe\utils.py", line 195, in raise_on_meta_error
    yield
  File "c:\Users\fjn197\Miniconda3\envs\datascience\lib\site-packages\dask\dataframe\core.py", line 6608, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "c:\Users\fjn197\Miniconda3\envs\datascience\lib

In [13]:
# Step 2: Convert your Pandas dataframes to Dask dataframes
admission_dask = dd.from_pandas(admission, npartitions=10)
concept_dask = dd.from_pandas(concepts, npartitions=10)

# Step 3: Repartition the dataframes by 'PID'
admission_dask = admission_dask.set_index('PID').sort_index()
concept_dask = concept_dask.set_index('PID').sort_index()
admission_dask

AttributeError: 'DataFrame' object has no attribute 'sort_index'

In [12]:


# Define a function to assign ADMISSION_ID to a partition
def assign_admission_id(concept_partition, admission_partition):
    for idx, row in concept_partition.iterrows():
        matching_admission = admission_partition[(admission_partition['TIMESTAMP'] <= row['TIMESTAMP']) & 
                                                 (admission_partition['TIMESTAMP_END'] >= row['TIMESTAMP'])]
        if not matching_admission.empty:
            concept_partition.loc[idx, 'ADMISSION_ID'] = matching_admission.iloc[0]['ADMISSION_ID']
    return concept_partition

# Step 4: Apply the function to each partition of the concept dataframe
concept_dask = concept_dask.map_partitions(assign_admission_id, admission_dask, meta=concept)

# Step 5: Compute the result to get the final dataframe
concept_with_admission_id = concept_dask.compute()

ValueError: The columns in the computed data do not match the columns in the provided metadata
  Extra:   []
  Missing: ['PID']

In [138]:
admission[admission['PID'].isin(concept['PID'].unique())].sort_values(by=['PID','TIMESTAMP'])

,ADMISSION_ID,TIMESTAMP,TIMESTAMP_END,PID
16990,fe4c527b-242b-19f7-5e29-97ac15ff7b35-1,1939-10-29 13:03:18+00:00,1939-10-29 13:03:18+00:00,fe4c527b-242b-19f7-5e29-97ac15ff7b35
17101,fe4c527b-242b-19f7-5e29-97ac15ff7b35-2,1940-03-31 14:06:00+00:00,1940-03-31 14:06:00+00:00,fe4c527b-242b-19f7-5e29-97ac15ff7b35
17212,fe4c527b-242b-19f7-5e29-97ac15ff7b35-3,1941-05-23 10:49:56+00:00,1941-05-23 10:49:56+00:00,fe4c527b-242b-19f7-5e29-97ac15ff7b35
17244,fe4c527b-242b-19f7-5e29-97ac15ff7b35-4,1946-06-21 10:49:56+00:00,1946-06-21 10:49:56+00:00,fe4c527b-242b-19f7-5e29-97ac15ff7b35
17255,fe4c527b-242b-19f7-5e29-97ac15ff7b35-5,1946-07-21 10:49:56+00:00,1946-07-21 10:49:56+00:00,fe4c527b-242b-19f7-5e29-97ac15ff7b35
...,...,...,...,...
17314,ff8dec86-a129-7e34-215a-8f34cd1a65e8-5,2015-07-02 04:44:27+00:00,2015-07-02 04:44:27+00:00,ff8dec86-a129-7e34-215a-8f34cd1a65e8
17315,ff8dec86-a129-7e34-215a-8f34cd1a65e8-6,2016-06-26 05:20:54+00:00,2016-06-26 05:20:54+00:00,ff8dec86-a129-7e34-215a-8f34cd1a65e8
17316,ff8dec86-a129-7e34-215a-8f34cd1a65e8-7,2017-06-21 05:33:22+00:00,2017-06-21 05:33:22+00:00,ff8dec86-a129-7e34-215a-8f34cd1a65e8
17317,ff8dec86-a129-7e34-215a-8f34cd1a65e8-8,2019-10-22 04:00:22+00:00,2019-10-22 04:00:22+00:00,ff8dec86-a129-7e34-215a-8f34cd1a65e8


In [105]:
admission_dask = dd.from_pandas(admission, npartitions=10)
concept_dask = dd.from_pandas(concept, npartitions=10)
# Define a custom function to assign ADMISSION_ID
def assign_admission_id(row):
    admission_row = admission_dask[(admission_dask['PID'] == row['PID']) & 
                                   (admission_dask['TIMESTAMP'] <= row['TIMESTAMP']) & 
                                   (admission_dask['TIMESTAMP_END'] >= row['TIMESTAMP'])]
    if len(admission_row) > 0:
        return admission_row['ADMISSION_ID'].values[0]
    else:
        return None

# Apply the function to the concept dataframe
concept_dask['ADMISSION_ID'] = concept_dask.apply(assign_admission_id, axis=1, meta=('ADMISSION_ID', 'object')).compute()

ValueError: Arrays chunk sizes are unknown: (nan,)

A possible solution: https://docs.dask.org/en/latest/array-chunks.html#unknown-chunks
Summary: to compute chunks sizes, use

   x.compute_chunk_sizes()  # for Dask Array `x`
   ddf.to_dask_array(lengths=True)  # for Dask DataFrame `ddf`

In [19]:
data_path = "C:\\Users\\fjn197\\PhD\\projects\\PHAIR\\pipelines\\synthea\\output\\synthea500\\csv"

In [24]:
patients = dd.read_csv(join(data_path, 'patients.csv'), blocksize=100)
conditions = dd.read_csv(join(data_path, 'conditions.csv'), blocksize=1000)

In [29]:
dd.read_csv(join(data_path, 'patients.csv'), blocksize=100, usecols=[0,1], names=['a','b'], header=0).head(10)

c:\Users\fjn197\Miniconda3\envs\datascience\lib\site-packages\dask\dataframe\core.py:7797: UserWarning: Insufficient elements for `head`. 10 elements requested, only 0 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


,a,b


In [22]:
unique_patients = patients['Id'].unique().compute()

In [ ]:
# get batches of unique patients in size batch_size
batch_size = 100
batches = [unique_patients[i:i + batch_size] for i in range(0, len(unique_patients), batch_size)]


In [23]:
for batch in 

0      dabfdec7-929a-3269-fd84-ab7e6ecaa245
1      6252ef78-e442-3081-f63b-36435c505a7f
2      a71c549b-c5fe-80b5-a1e8-4741bd43aeb9
3      6b7b3fc8-b581-0e0c-59f4-ffcd6337d687
4      a793a189-7e7f-48b6-ea63-43036cef8d3d
                       ...                 
560    a46a026e-f05c-1cee-e7e5-ce70c24ac2b0
561    ae7e6ac5-ffaa-e982-6774-801bc2a9924e
562    2c6a19fc-d4b6-42d4-4a85-f3cff500ffac
563    82b8b06f-1177-e894-f7fd-63c3244b89af
564    a5aad816-cf2e-0a5e-1886-e249ef979ce5
Name: Id, Length: 565, dtype: object

In [27]:
formatted_data_path = "C:\\Users\\fjn197\\PhD\\projects\\PHAIR\\pipelines\\ehr_preprocess\\formatted_data\\synthea500"
patients = pd.read_csv(join(formatted_data_path, 'patients_info.csv'))
patients

,PID,DATE_OF_BIRTH,DATE_OF_DEATH,RACE,ETHNICITY,GENDER
0,6252ef78-e442-3081-f63b-36435c505a7f,1991-06-21,NaN,white,nonhispanic,M
1,a71c549b-c5fe-80b5-a1e8-4741bd43aeb9,2004-04-26,NaN,white,nonhispanic,F
2,6b7b3fc8-b581-0e0c-59f4-ffcd6337d687,1985-04-06,NaN,white,nonhispanic,M
3,a793a189-7e7f-48b6-ea63-43036cef8d3d,1997-04-28,NaN,white,nonhispanic,M
4,78a7ab19-bdd0-2238-e4e5-de757ee074d1,2001-06-01,NaN,white,nonhispanic,M
...,...,...,...,...,...,...
559,a46a026e-f05c-1cee-e7e5-ce70c24ac2b0,1966-11-28,NaN,other,hispanic,M
560,ae7e6ac5-ffaa-e982-6774-801bc2a9924e,1977-05-13,NaN,asian,nonhispanic,M
561,2c6a19fc-d4b6-42d4-4a85-f3cff500ffac,1951-10-10,NaN,white,nonhispanic,F
562,82b8b06f-1177-e894-f7fd-63c3244b89af,1980-07-21,NaN,white,nonhispanic,M
